In [44]:
import pandas as pd

# Load the dataset (replace 'groceries.csv' with your actual file path)
file_path = 'groceries.csv'  # Update this with the correct path
groceries_data = pd.read_csv(file_path, delimiter=';')

# Combine item columns into a list for each transaction, excluding NaNs
groceries_data_cleaned = groceries_data.iloc[:, 1:].apply(lambda row: row.dropna().tolist(), axis=1)

# Convert to a DataFrame with a single column for transactions
transactions_df = pd.DataFrame({'Transaction': groceries_data_cleaned})

# Display the cleaned and restructured data
print(transactions_df.head())

# Save the cleaned data to a new CSV if needed
transactions_df.to_csv('cleaned_groceries.csv', index=False)


                                         Transaction
0  [citrus fruit, semi-finished bread, margarine,...
1                   [tropical fruit, yogurt, coffee]
2                                       [whole milk]
3    [pip fruit, yogurt, cream cheese, meat spreads]
4  [other vegetables, whole milk, condensed milk,...


In [67]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

# Load and clean the dataset
file_path = 'groceries.csv'  # Update with actual path
groceries_data = pd.read_csv(file_path, delimiter=';')

# Clean and restructure data
groceries_data_cleaned = groceries_data.iloc[:, 1:].apply(lambda row: row.dropna().tolist(), axis=1)

# Convert to one-hot encoding format
te = TransactionEncoder()
te_data = te.fit(groceries_data_cleaned).transform(groceries_data_cleaned)
groceries_encoded = pd.DataFrame(te_data, columns=te.columns_)


In [70]:
# Apriori - Find frequent itemsets and generate rules
frequent_itemsets_apriori = apriori(groceries_encoded, min_support=0.005, use_colnames=True)
rules_apriori = association_rules(frequent_itemsets_apriori, metric="lift", min_threshold=0.2,num_itemsets=num_itemsets)

# FP-Growth - Find frequent itemsets and generate rules
frequent_itemsets_fp = fpgrowth(groceries_encoded, min_support=0.005, use_colnames=True)
rules_fp = association_rules(frequent_itemsets_fp, metric="lift", min_threshold=0.2,num_itemsets=num_itemsets)


In [71]:
# Combine Apriori and FP-Growth rules
combined_rules = pd.concat([rules_apriori, rules_fp], ignore_index=True)

# Filter and display the best rules (lift > 1.5 and confidence > 0.2)
filtered_rules = combined_rules[(combined_rules['lift'] > 1.5) & (combined_rules['confidence'] > 0.2)]
print(filtered_rules.sort_values(by='lift', ascending=False).head())


                            antecedents                     consequents  \
3367          (root vegetables, yogurt)    (whole milk, tropical fruit)   
2957          (root vegetables, yogurt)    (whole milk, tropical fruit)   
2953  (tropical fruit, root vegetables)            (whole milk, yogurt)   
3363  (tropical fruit, root vegetables)            (whole milk, yogurt)   
2828       (pip fruit, root vegetables)  (other vegetables, whole milk)   

      antecedent support  consequent support   support  confidence      lift  \
3367            0.025826            0.042298  0.005694    0.220472  5.212371   
2957            0.025826            0.042298  0.005694    0.220472  5.212371   
2953            0.021047            0.056024  0.005694    0.270531  4.828814   
3363            0.021047            0.056024  0.005694    0.270531  4.828814   
2828            0.015557            0.074835  0.005491    0.352941  4.716272   

      representativity  leverage  conviction  zhangs_metric   jaccar

In [72]:
from sklearn.neighbors import NearestNeighbors

# KNN to recommend products
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(groceries_encoded.T)  # Transpose to focus on products

# Function to get recommended products
def get_recommended_products(product_name, n_neighbors=5):
    product_index = groceries_encoded.columns.get_loc(product_name)
    distances, indices = knn.kneighbors(groceries_encoded.T.iloc[product_index].values.reshape(1, -1), n_neighbors=n_neighbors)
    
    # Get the recommended products
    recommended_products = [groceries_encoded.columns[i] for i in indices.flatten()]
    return recommended_products

# Ask the user for a product name and recommend related products
product_name = input("Enter the product name: ")

if product_name in groceries_encoded.columns:
    recommended_products = get_recommended_products(product_name)
    print(f"Recommended items for '{product_name}': {recommended_products}")
else:
    print(f"Product '{product_name}' not found in the dataset.")


Recommended items for 'whole milk': ['whole milk', 'other vegetables', 'yogurt', 'root vegetables', 'rolls/buns']


In [73]:
# Combine Apriori and FP-Growth rules for better recommendations
def recommend_items(product_name, rules, top_n=5):
    product_name = product_name.lower()
    
    # Filter rules where the product appears in the antecedents
    filtered_rules = rules[rules['antecedents'].apply(lambda x: any(product_name in item for item in x))]
    
    recommendations = [
        ', '.join(consequent)  # Join all items in the consequents
        for _, row in filtered_rules.head(top_n).iterrows()
        for consequent in [row['consequents']]
    ]
    return recommendations

# Interactive recommendation
product_name = input("Enter the product name: ").strip()
recommendations = recommend_items(product_name, filtered_rules)

# Display recommendations from association rules
if recommendations:
    print(f"Recommended items for '{product_name}' based on association rules:")
    for item in recommendations:
        print(f"- {item}")
else:
    print(f"No recommendations found for '{product_name}' based on association rules.")
    
# Now, use KNN to get product recommendations
if product_name in groceries_encoded.columns:
    recommended_products_knn = get_recommended_products(product_name)
    print(f"\nRecommended items for '{product_name}' based on KNN:")
    for product in recommended_products_knn:
        print(f"- {product}")
else:
    print(f"Product '{product_name}' not found in the dataset for KNN recommendations.")


Recommended items for 'pip fruit' based on association rules:
- other vegetables
- root vegetables
- tropical fruit
- whole milk
- yogurt

Recommended items for 'pip fruit' based on KNN:
- pip fruit
- tropical fruit
- whole milk
- other vegetables
- yogurt


In [59]:
from mlxtend.frequent_patterns import apriori, association_rules

# Convert transactions to a one-hot encoded DataFrame
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_data = te.fit(transactions_df['Transaction']).transform(transactions_df['Transaction'])
transactions_encoded = pd.DataFrame(te_data, columns=te.columns_)

# Apply Apriori to find frequent itemsets
frequent_itemsets = apriori(transactions_encoded, min_support=0.005, use_colnames=True)

# Calculate the total number of itemsets
num_itemsets = len(frequent_itemsets)

# Generate association rules
rules = association_rules(frequent_itemsets, num_itemsets=num_itemsets, metric="lift", min_threshold=0.2)

# Display the rules
print(rules.head())


          antecedents         consequents  antecedent support  \
0     (bottled water)          (UHT-milk)            0.110524   
1          (UHT-milk)     (bottled water)            0.033452   
2  (other vegetables)          (UHT-milk)            0.193493   
3          (UHT-milk)  (other vegetables)            0.033452   
4        (rolls/buns)          (UHT-milk)            0.183935   

   consequent support   support  confidence      lift  representativity  \
0            0.033452  0.007321    0.066237  1.980074               1.0   
1            0.110524  0.007321    0.218845  1.980074               1.0   
2            0.033452  0.008134    0.042039  1.256694               1.0   
3            0.193493  0.008134    0.243161  1.256694               1.0   
4            0.033452  0.006406    0.034826  1.041071               1.0   

   leverage  conviction  zhangs_metric   jaccard  certainty  kulczynski  
0  0.003624    1.035111       0.556472  0.053571   0.033920    0.142541  
1  0.00362

In [46]:
# Filter rules with lift > 1.5 and confidence > 0.2
filtered_rules = rules[(rules['lift'] > 1.5) & (rules['confidence'] > 0.2)]

# Display the top rules
print(filtered_rules.sort_values(by='lift', ascending=False).head())


          antecedents         consequents  antecedent support  \
5            (yogurt)        (whole milk)            0.139502   
4        (whole milk)            (yogurt)            0.255516   
0  (other vegetables)        (whole milk)            0.193493   
1        (whole milk)  (other vegetables)            0.255516   

   consequent support   support  confidence      lift  representativity  \
5            0.255516  0.056024    0.401603  1.571735               1.0   
4            0.139502  0.056024    0.219260  1.571735               1.0   
0            0.255516  0.074835    0.386758  1.513634               1.0   
1            0.193493  0.074835    0.292877  1.513634               1.0   

   leverage  conviction  zhangs_metric   jaccard  certainty  kulczynski  
5  0.020379    1.244132       0.422732  0.165267   0.196226    0.310432  
4  0.020379    1.102157       0.488608  0.165267   0.092688    0.310432  
0  0.025394    1.214013       0.420750  0.200000   0.176286    0.339817  
1 

In [60]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets, num_itemsets=num_itemsets, metric="lift", min_threshold=0.2)

# Save association rules to CSV
rules.to_csv('association_rules.csv', index=False)

# Display the rules
print(rules.head())

          antecedents         consequents  antecedent support  \
0     (bottled water)          (UHT-milk)            0.110524   
1          (UHT-milk)     (bottled water)            0.033452   
2  (other vegetables)          (UHT-milk)            0.193493   
3          (UHT-milk)  (other vegetables)            0.033452   
4        (rolls/buns)          (UHT-milk)            0.183935   

   consequent support   support  confidence      lift  representativity  \
0            0.033452  0.007321    0.066237  1.980074               1.0   
1            0.110524  0.007321    0.218845  1.980074               1.0   
2            0.033452  0.008134    0.042039  1.256694               1.0   
3            0.193493  0.008134    0.243161  1.256694               1.0   
4            0.033452  0.006406    0.034826  1.041071               1.0   

   leverage  conviction  zhangs_metric   jaccard  certainty  kulczynski  
0  0.003624    1.035111       0.556472  0.053571   0.033920    0.142541  
1  0.00362

In [48]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

# Apply FP-Growth to find frequent itemsets
frequent_itemsets_fp = fpgrowth(transactions_encoded, min_support=0.01, use_colnames=True)

# Calculate the total number of frequent itemsets
num_itemsets_fp = len(frequent_itemsets_fp)

# Print the number of itemsets
print(f"Number of frequent itemsets (FP-Growth): {num_itemsets_fp}")

# Generate association rules from the FP-Growth frequent itemsets
rules_fp = association_rules(frequent_itemsets_fp, metric="lift", min_threshold=0.05,num_itemsets=num_itemsets_fp)

# Display the rules from FP-Growth
print(rules_fp.head())

# Filter rules with lift > 1.5 and confidence > 0.2
filtered_rules_fp = rules_fp[(rules_fp['lift'] > 1.5) & (rules_fp['confidence'] > 0.2)]

# Display the top rules from FP-Growth
print(filtered_rules_fp.sort_values(by='lift', ascending=False).head())

# Save the FP-Growth association rules to CSV
filtered_rules_fp.to_csv('filtered_fp_growth_association_rules.csv', index=False)


Number of frequent itemsets (FP-Growth): 333
        antecedents     consequents  antecedent support  consequent support  \
0      (whole milk)  (citrus fruit)            0.255516            0.082766   
1    (citrus fruit)    (whole milk)            0.082766            0.255516   
2          (yogurt)  (citrus fruit)            0.139502            0.082766   
3    (citrus fruit)        (yogurt)            0.082766            0.139502   
4  (tropical fruit)  (citrus fruit)            0.104931            0.082766   

    support  confidence      lift  representativity  leverage  conviction  \
0  0.030503    0.119379  1.442377               1.0  0.009355    1.041577   
1  0.030503    0.368550  1.442377               1.0  0.009355    1.179008   
2  0.021657    0.155248  1.875752               1.0  0.010111    1.085803   
3  0.021657    0.261671  1.875752               1.0  0.010111    1.165467   
4  0.019929    0.189922  2.294702               1.0  0.011244    1.132280   

   zhangs_metric 

In [61]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Apply TransactionEncoder to transform the data
te = TransactionEncoder()
te_data = te.fit(transactions_df['Transaction']).transform(transactions_df['Transaction'])

# Convert to DataFrame
transactions_encoded = pd.DataFrame(te_data, columns=te.columns_)

# Apply Apriori to find frequent itemsets
frequent_itemsets = apriori(transactions_encoded, min_support=0.005, use_colnames=True)

# Generate association rules from the frequent itemsets
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.2,num_itemsets=num_itemsets)

# Display the rules
print(rules.head())

# Filter rules with lift > 1.5 and confidence > 0.2
filtered_rules = rules[(rules['lift'] > 1.5) & (rules['confidence'] > 0.2)]

# Display the top rules
print(filtered_rules.sort_values(by='lift', ascending=False).head())

# Save the association rules to CSV
filtered_rules.to_csv('association_rules.csv', index=False)


          antecedents         consequents  antecedent support  \
0     (bottled water)          (UHT-milk)            0.110524   
1          (UHT-milk)     (bottled water)            0.033452   
2  (other vegetables)          (UHT-milk)            0.193493   
3          (UHT-milk)  (other vegetables)            0.033452   
4        (rolls/buns)          (UHT-milk)            0.183935   

   consequent support   support  confidence      lift  representativity  \
0            0.033452  0.007321    0.066237  1.980074               1.0   
1            0.110524  0.007321    0.218845  1.980074               1.0   
2            0.033452  0.008134    0.042039  1.256694               1.0   
3            0.193493  0.008134    0.243161  1.256694               1.0   
4            0.033452  0.006406    0.034826  1.041071               1.0   

   leverage  conviction  zhangs_metric   jaccard  certainty  kulczynski  
0  0.003624    1.035111       0.556472  0.053571   0.033920    0.142541  
1  0.00362

In [50]:
import mlxtend
print(mlxtend.__version__)


0.23.3


In [116]:
import pandas as pd

# Step 1: Load your dataset
file_path = 'groceries.csv'  # Replace with your actual file path
groceries_data = pd.read_csv(file_path, delimiter=';')

# Step 2: Cleaning and transforming the data
# We're assuming that each row represents a transaction, and items are in columns starting from the 2nd column
groceries_data_cleaned = groceries_data.iloc[:, 1:].apply(lambda row: row.dropna().tolist(), axis=1)

# Step 3: Let's assume each transaction corresponds to a "userId" and the items represent the products purchased.
# For simplicity, we'll consider that we have one "user" per transaction. You can adjust this if you have more complex data.

# Number of transactions = n_ratings
n_transactions = len(groceries_data)

# Number of unique items
n_items = len(groceries_data.iloc[:, 1:].stack().unique())

# Number of unique users (for simplicity, we'll consider each transaction as from a unique user)
n_users = n_transactions  # If each row represents a unique transaction/user

# Print the information
print(f"Number of transactions: {n_transactions}")
print(f"Number of unique items: {n_items}")
print(f"Number of unique users: {n_users}")
print(f"Average transactions per user: {round(n_transactions/n_users, 2)}")
print(f"Average items per transaction: {round(n_items/n_transactions, 2)}")


Number of transactions: 9835
Number of unique items: 169
Number of unique users: 9835
Average transactions per user: 1.0
Average items per transaction: 0.02


In [53]:
# Create a user-item matrix where each row represents a transaction and each column an item
te = TransactionEncoder()
te_data = te.fit(transactions_df['Transaction']).transform(transactions_df['Transaction'])
user_item_matrix = pd.DataFrame(te_data, columns=te.columns_)

# Display the user-item matrix
print(user_item_matrix.head())


   Instant food products  UHT-milk  abrasive cleaner  artif. sweetener  \
0                  False     False             False             False   
1                  False     False             False             False   
2                  False     False             False             False   
3                  False     False             False             False   
4                  False     False             False             False   

   baby cosmetics  baby food   bags  baking powder  bathroom cleaner   beef  \
0           False      False  False          False             False  False   
1           False      False  False          False             False  False   
2           False      False  False          False             False  False   
3           False      False  False          False             False  False   
4           False      False  False          False             False  False   

   ...  turkey  vinegar  waffles  whipped/sour cream  whisky  white bread  \
0  

In [55]:
groceries_data_cleaned = groceries_data.iloc[:, 1:].apply(lambda row: row.dropna().str.lower().tolist(), axis=1)


In [56]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Create the user-item matrix
user_item_matrix = transactions_df['Transaction'].apply(pd.Series)\
    .stack()\
    .reset_index(level=1, drop=True)\
    .reset_index()\
    .rename(columns={0: 'Item', 'index': 'TransactionID'})\
    .pivot_table(index='TransactionID', columns='Item', aggfunc='size', fill_value=0)

# Convert to a NumPy array for KNN
user_item_array = user_item_matrix.values

print("User-Item Matrix Shape:", user_item_array.shape)


User-Item Matrix Shape: (9835, 169)


In [57]:
# Initialize the KNN model
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)

# Fit the model using the user-item matrix
knn.fit(user_item_array)

# Find the 5 nearest neighbors for each user
distances, indices = knn.kneighbors(user_item_array, n_neighbors=5)

print("Distances:\n", distances[:5])
print("Indices:\n", indices[:5])


Distances:
 [[0.         0.29289322 0.29289322 0.42264973 0.42264973]
 [0.         0.1339746  0.18350342 0.22540333 0.33333333]
 [0.         0.         0.         0.         0.        ]
 [0.         0.29289322 0.29289322 0.29289322 0.33333333]
 [0.         0.29289322 0.29289322 0.29289322 0.29289322]]
Indices:
 [[   0 7219 8837 6882 3613]
 [   1 3636 4094 7099 8999]
 [1786 8765 7633 4164 8513]
 [   3 4687 2871 2384 5332]
 [   4 5485 6343 8526 2899]]


In [24]:
print(groceries_data.head())
print(groceries_data_cleaned.head())


   Trans            Item 1               Item 2          Item 3  \
0      1      citrus fruit  semi-finished bread       margarine   
1      2    tropical fruit               yogurt          coffee   
2      3        whole milk                  NaN             NaN   
3      4         pip fruit               yogurt    cream cheese   
4      5  other vegetables           whole milk  condensed milk   

                     Item 4 Item 5 Item 6 Item 7 Item 8 Item 9  ... Item 23  \
0               ready soups    NaN    NaN    NaN    NaN    NaN  ...     NaN   
1                       NaN    NaN    NaN    NaN    NaN    NaN  ...     NaN   
2                       NaN    NaN    NaN    NaN    NaN    NaN  ...     NaN   
3              meat spreads    NaN    NaN    NaN    NaN    NaN  ...     NaN   
4  long life bakery product    NaN    NaN    NaN    NaN    NaN  ...     NaN   

  Item 24 Item 25 Item 26 Item 27 Item 28 Item 29 Item 30 Item 31 Item 32  
0     NaN     NaN     NaN     NaN     NaN     

recommendation system on fpgrowth algrithm

In [79]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Load the dataset
file_path = 'groceries.csv'  # Update with your actual file path
groceries_data = pd.read_csv(file_path, delimiter=';')

# Clean and transform the data into a list of transactions
groceries_data_cleaned = groceries_data.iloc[:, 1:].apply(lambda row: row.dropna().tolist(), axis=1)

# Apply TransactionEncoder to convert transactions into one-hot encoded format
te = TransactionEncoder()
te_data = te.fit(groceries_data_cleaned).transform(groceries_data_cleaned)
groceries_encoded = pd.DataFrame(te_data, columns=te.columns_)

# Apply FP-Growth with lower support threshold
frequent_itemsets_fp = fpgrowth(groceries_encoded, min_support=0.003, use_colnames=True)

# Generate association rules from FP-Growth frequent itemsets
rules_fp = association_rules(frequent_itemsets_fp, metric="lift", min_threshold=0.05,num_itemsets=num_itemsets)

# Filter the rules based on relaxed thresholds
filtered_rules_fp = rules_fp[(rules_fp['lift'] > 1.0) & (rules_fp['confidence'] > 0.2)]

# Function to recommend items based on FP-Growth association rules
def recommend_items_fp(product_name, rules_fp, top_n=5):
    product_name = product_name.lower()

    # Filter rules where the product is in the antecedents
    recommendations = []
    for _, row in filtered_rules_fp.iterrows():
        if product_name in row['antecedents']:
            recommendations.extend(row['consequents'])

    # Return the top N recommendations
    return list(set(recommendations))[:top_n]

# Get user input for product name
product_name = input("Enter the product name for FP-Growth recommendations: ").strip()

# Get recommendations from FP-Growth
recommendations_fp = recommend_items_fp(product_name, filtered_rules_fp)

# Display results for FP-Growth
print(f"\n--- Recommendations based on FP-Growth Algorithm ---")
if recommendations_fp:
    for i, rec in enumerate(recommendations_fp, 1):
        print(f"{i}. {rec}")
else:
    print("No recommendations found using FP-Growth.")



--- Recommendations based on FP-Growth Algorithm ---
1. whipped/sour cream
2. soda
3. curd
4. whole milk
5. sausage


recommendation system based on apriori algorithm

In [102]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Load the dataset (replace 'groceries.csv' with your actual file path)
file_path = 'groceries.csv'  # Update with your actual file path
groceries_data = pd.read_csv(file_path, delimiter=';')

# Clean and transform the data into a list of transactions (excluding NaNs)
groceries_data_cleaned = groceries_data.iloc[:, 1:].apply(lambda row: row.dropna().tolist(), axis=1)

# Apply TransactionEncoder to convert transactions into one-hot encoded format
te = TransactionEncoder()
te_data = te.fit(groceries_data_cleaned).transform(groceries_data_cleaned)
groceries_encoded = pd.DataFrame(te_data, columns=te.columns_)

# Apply Apriori - Find frequent itemsets with minimum support
frequent_itemsets = apriori(groceries_encoded, min_support=0.005, use_colnames=True)

# Generate association rules from Apriori
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.2,num_itemsets=num_itemsets)

# Function to recommend items based on Apriori association rules
def recommend_items_apriori(product_name, rules, top_n=5):
    product_name = product_name.lower()

    # Filter rules where the product is in the antecedents
    recommendations = []
    for _, row in rules.iterrows():
        if product_name in row['antecedents']:
            recommendations.extend(row['consequents'])

    # Return the top N recommendations
    return list(set(recommendations))[:top_n]

# Get user input for product name
product_name = input("Enter the product name for Apriori recommendations: ").strip().lower()

# Get recommendations from Apriori
recommendations_apriori = recommend_items_apriori(product_name, rules)

# Display results for Apriori
print(f"\n--- Recommendations based on Apriori Algorithm ---")
if recommendations_apriori:
    for i, rec in enumerate(recommendations_apriori, 1):
        print(f"{i}. {rec}")
else:
    print("No recommendations found using Apriori.")



--- Recommendations based on Apriori Algorithm ---
1. coffee
2. pork
3. whole milk
4. frankfurter
5. root vegetables


In [89]:
print("Shape of user_item_matrix:", user_item_matrix.shape)


Shape of user_item_matrix: (9835, 169)


In [100]:
from sklearn.neighbors import NearestNeighbors

# KNN to recommend products using the user-item matrix (groceries_encoded)
knn = NearestNeighbors(metric='cosine', algorithm='brute')

# Transpose to focus on products (columns) rather than transactions (rows)
knn.fit(groceries_encoded.T)

# Function to get recommended products based on a product name
def get_recommended_products(product_name, groceries_encoded, n_neighbors=5):
    # Check if the product exists in the dataset
    if product_name not in groceries_encoded.columns:
        print(f"Product '{product_name}' not found in the dataset.")
        return []

    # Get the index of the product in the dataset
    product_index = groceries_encoded.columns.get_loc(product_name)
    
    # Use the KNN model to find similar products
    distances, indices = knn.kneighbors(groceries_encoded.T.iloc[product_index].values.reshape(1, -1), n_neighbors=n_neighbors)
    
    # Get the recommended product names
    recommended_products = [groceries_encoded.columns[i] for i in indices.flatten()]
    return recommended_products

# Ask the user for a product name and recommend related products
product_name = input("Enter the product name: ").strip().lower()

# Get the recommendations
recommended_products = get_recommended_products(product_name, groceries_encoded)

# Display the recommendations
if recommended_products:
    print(f"Recommended items for '{product_name}':")
    for i, product in enumerate(recommended_products, 1):
        print(f"{i}. {product}")
else:
    print("No recommendations found.")


Recommended items for 'pip fruit':
1. pip fruit
2. tropical fruit
3. whole milk
4. other vegetables
5. yogurt
